![alt](https://research.utm.my/wp-content/uploads/sites/26/2022/06/logo-300x122.png)
# Center for Artificial Intelligence and Robotics
### Universiti Teknologi Malaysia


#### Classification Inference - Resnet50

*Author: Dr. Ibrahim, Azzam, Thaqif & Syahmi*

In [ ]:
import torch
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.eval()

All pre-trained models expect input images normalized in the same way,
i.e. mini-batches of 3-channel RGB images of shape `(3 x H x W)`, where `H` and `W` are expected to be at least `224`.
The images have to be loaded in to a range of `[0, 1]` and then normalized using `mean = [0.485, 0.456, 0.406]`
and `std = [0.229, 0.224, 0.225]`.

Here's a sample execution.

In [ ]:
# Download an example image from the pytorch website
# import urllib
# url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
# try: urllib.URLopener().retrieve(url, filename)
# except: urllib.request.urlretrieve(url, filename)
# Load the image
filename = "test_images/cat_1.jpeg"
input_image = Image.open(filename)

In [ ]:
# Preprocess the image
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model

# Move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

# Perform inference
with torch.no_grad():
    output = model(input_batch)
    
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
# Get probabilities
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

In [ ]:
# Download and read ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [ ]:
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

# Display the image after predictions
plt.imshow(input_image)
plt.axis('off')  # Hide the axis
plt.show()

### Model Description

ResNet-50 is a deep residual network that has 50 layers. It was introduced in the paper "Deep Residual Learning for Image Recognition" and became one of the most widely used models in computer vision tasks.

### References

 - [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)


# Plant Classification with ResNet-50

This section demonstrates how to load a custom-trained ResNet-50 model on a Jetson device, and use it for plant classification. The model is configured to classify images into three categories.

## Sections

1. Prerequisites
2. Import Necessary Libraries
3. Set Up the Device
4. Model Architecture
5. Load the Model Weights
6. Prediction Function
    - Function Definition
    - Usage
7. Example Usage
8. Conclusion
   

In [ ]:
# Import necessery libraries
import torch
import torch.nn as nn
from torchvision.models import resnet50
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Ensure you are using the correct device (GPU on Jetson)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model architecture
model = resnet50(pretrained=False)  # Set pretrained to False since we'll load custom weights
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # Adjust the output layer to match the number of classes

# Load the model weights
model_load_path = '___' # FIX ME # Replace with your model path
model.load_state_dict(torch.load(model_load_path, map_location=device))
model = model.to(device)
model.eval()  # Set the model to evaluation mode

print("Model loaded successfully.")

# Prediction function (reuse from earlier)
def predict_image(model, image_path, class_names):
    
    # Transform to match the training preprocessing
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image)
    image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension

    # Move the image to the device
    image_tensor = image_tensor.to(device)

    # Make prediction
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        probabilities = torch.nn.functional.softmax(output, dim=1).cpu().numpy().squeeze()
        predicted_idx = probabilities.argmax()
    
    predicted_class = class_names[predicted_idx]
    confidence = probabilities[predicted_idx] * 100
    result = f"This image most likely belongs to {predicted_class} with a {confidence:.2f} percent confidence."
    
    # Display the image
    plt.imshow(image)
    plt.title(result)
    plt.axis('off')
    plt.show()
    
    return result, dict(zip(class_names, probabilities))

Here’s how you can use the predict_image function with actual class names and an image:

In [ ]:
# Example usage:
class_names = ['___', '___', '___'] 
image_path = 'testbetiktree.jpg'
result, probabilities = predict_image(model, image_path, class_names)
print(result)
print(f"Probabilities: {probabilities}")

## Conclusion
This notebook provides a template for loading and using a custom-trained ResNet-50 model on Jetson devices. Modify the code to suit your specific use case, such as adjusting the number of output classes or updating the image preprocessing steps.